In [49]:
# coding: utf-8

import json
import requests
import numpy as np


class DistributInfo(object):
    # projectParam - distributInfo - 分散式信息
    # 分散式开关站个数
    def __init__(self, switchStation35kVNumber=0, switchStation10kVNumber=0, competition=False):
        self.competition = competition
        if competition:
            # 竞价设置为null
            self.switchStation35kVNumber = None
            self.switchStation10kVNumber = None
        else:
            self.switchStation35kVNumber = switchStation35kVNumber
            self.switchStation10kVNumber = switchStation10kVNumber

    def keys(self):
        '''当对实例化对象使用dict(obj)的时候, 会调用这个方法,这里定义了字典的键, 其对应的值将以obj['name']的形式取,
        但是对象是不可以以这种方式取值的, 为了支持这种取值, 可以为类增加一个方法'''
        return ('switchStation35kVNumber', 'switchStation10kVNumber')

    def __getitem__(self, item):
        '''内置方法, 当使用obj['name']的形式的时候, 将调用这个方法, 这里返回的结果就是值'''
        return getattr(self, item)


class PriceInfo(object):
    # projectParam - priceInfo - 价格信息
    def __init__(self, steelBarUnitPrice=5800.0, towerUnitPrice=9000.0,
                 concreteCushionUnitPrice=522.5, anchorUnitPrice=12000.0,
                 concreteUnitPrice=646.9):
        self.steelBarUnitPrice = steelBarUnitPrice  # 钢筋单价
        self.towerUnitPrice = towerUnitPrice  # 塔筒单价
        self.concreteCushionUnitPrice = concreteCushionUnitPrice  # 基础垫层混凝土（单价）
        self.anchorUnitPrice = anchorUnitPrice  # 锚栓单价
        self.concreteUnitPrice = concreteUnitPrice  # 基础混凝土（单价）

    def keys(self):
        return ('steelBarUnitPrice', 'towerUnitPrice', 'concreteCushionUnitPrice',
                'anchorUnitPrice', 'concreteUnitPrice')

    def __getitem__(self, item):
        return getattr(self, item)


class FinancingInfo(object):
    # projectParam - financingInfo -融资信息
    def __init__(self, loanInterestRateWithTax=0.049, financing="LONG_TERM_BANK_BORROWING",
                 longtermFinancingGraceYears=1, graceInterestRateWithTax=0.049,
                 interestTaxDeduction=False, returnLoanMethod="AVERAGE_CAPITAL",
                 longtermFinancingYears=15):
        self.loanInterestRateWithTax = loanInterestRateWithTax  # 贷款期利率（含税）
        self.financing = financing  # 融资方式：银行长期借款、融资租赁
        self.longtermFinancingGraceYears = longtermFinancingGraceYears  # 宽限期
        self.graceInterestRateWithTax = graceInterestRateWithTax  # 宽限期利率（含税）
        self.interestTaxDeduction = interestTaxDeduction  # 利息税是否可以抵扣
        self.returnLoanMethod = returnLoanMethod  # 还款方式：等额本金、等额本息
        self.longtermFinancingYears = longtermFinancingYears  # 期限（含宽限期）

    def keys(self):
        return ('loanInterestRateWithTax', 'financing', 'longtermFinancingGraceYears',
                'graceInterestRateWithTax', 'interestTaxDeduction', 'returnLoanMethod', 'longtermFinancingYears')

    def __getitem__(self, item):
        return getattr(self, item)


class ProjectInfo(object):
    # projectParam - projectInfo - 项目信息
    def __init__(self, province="冀北", irrType="CAPITAL_IRR", name="中广核商毅二期50MW风电项目",
                 propertiesType="TYPICAL", distribut=False, irrValue=0.1, capacity=50.0):
        self.province = province  # 所在区域 其实也应当是枚举，但是暂无力处理，这个值用于根据地区去获取
        self.irrType = irrType  # 全投资IRR  FI_IRR、资本金IRR
        self.name = name
        self.propertiesType = propertiesType  # 参数规则 就是各种配置文件 中广核、可研之类的
        self.distribut = distribut  # 是分散式！！！
        self.irrValue = irrValue  # 投资基准
        self.capacity = capacity  # 容量 单位MW

    def keys(self):
        return ('province', 'irrType', 'name',
                'propertiesType', 'distribut', 'irrValue', 'capacity')

    def __getitem__(self, item):
        return getattr(self, item)


class OperationInfo(object):
    # projectParam - operationInfo - 代运维信息
    def __init__(self, enable=False, outWarrantyFee=136.5, warrantyFee=52.5,
                 warrantyPeriod=5, competition=False):
        self.competition = competition
        self.enable = enable
        if competition:
            self.enable = False  # 代运维启用   #竞价设置为false
            self.outWarrantyFee = None  # 竞价设置为null
            self.warrantyFee = None  # 竞价设置为null
            self.warrantyPeriod = None  # 竞价设置为null
        else:
            if enable:
                self.outWarrantyFee = outWarrantyFee  # 质保外费用
                self.warrantyFee = warrantyFee  # 质保期费用
                self.warrantyPeriod = warrantyPeriod  # 质保年限

    def keys(self):
        if self.enable | self.competition:
            return ('enable', 'outWarrantyFee', 'warrantyFee',
                    'warrantyPeriod')
        else:
            return ('enable',)

    def __getitem__(self, item):
        return getattr(self, item)


class EngineeringInfo(object):
    # projectParam - engineeringInfo - 工程信息
    def __init__(self, apportioningInvestment=0.0, periodMonth=0, terrain="PLAIN",
                 newBoosterStation=True):
        self.apportioningInvestment = apportioningInvestment  # 分摊投资 万元 与可研信息互斥
        self.periodMonth = periodMonth  # 工期 月
        self.terrain = terrain  # 地形 平原、山地、丘陵、复杂山地
        self.newBoosterStation = newBoosterStation  # 是否新建升压站

    def keys(self):
        return ('apportioningInvestment', 'periodMonth', 'terrain',
                'newBoosterStation')

    def __getitem__(self, item):
        return getattr(self, item)


class MarketInfo(object):
    # projectParam - marketInfo - 市场信息
    def __init__(self, restrictYearNumber=20, tradingPrice=0.0, tradingRatio=0.0,
                 feedinTariffs=0.45, restrictLimitElecRatio=0.07,
                 tradingYearNumber=20):
        self.restrictYearNumber = restrictYearNumber  # 限电年限 年数
        self.tradingPrice = tradingPrice  # 市场交易电价,单位元/kWh
        self.tradingRatio = tradingRatio  # 市场交易电量比例 %，同样，给数值
        self.feedinTariffs = feedinTariffs  # 常规模式上网电价（含税）元/kWh
        self.restrictLimitElecRatio = restrictLimitElecRatio  # 限电率 7%填0.07
        self.tradingYearNumber = tradingYearNumber  # 市场交易年数

    def keys(self):
        return ('restrictYearNumber', 'tradingPrice', 'tradingRatio',
                'feedinTariffs', 'restrictLimitElecRatio', 'tradingYearNumber')

    def __getitem__(self, item):
        return getattr(self, item)


class FeasibleInfo(object):
    # projectParam - feasibleInfo - 可研信息
    def __init__(self, enable=False, competition=False, turbineNumber=23, baseConcretePrice=733.0,
                 baseCushionConcretePrice=35.0, baseSteelBarPrice=520.0,
                 staticTotalInvestment=24000, totalBasePrice=1500,
                 totalCollectorLinePrice=560.0, totalConstructionInstallationPlatformPrice=66.0,
                 totalFanPrice=16100.0, totalInnerRoadPrice=430.0,
                 totalPackageSubstationPrice=470.0, totalTowerPrice=4600.0,
                 totalTurbineAndTowerInstallationPrice=47.0):
        self.enable = enable  # 是否有这个类
        self.turbineNumber = turbineNumber  # 风机台数
        self.competition = competition
        if enable | competition:
            self.baseConcretePrice = baseConcretePrice  # 基础混凝土 万元
            self.baseCushionConcretePrice = baseCushionConcretePrice  # 基础垫层混凝土 万元
            self.baseSteelBarPrice = baseSteelBarPrice  # 基础钢筋 万元
            self.staticTotalInvestment = staticTotalInvestment  # 静态总投资 万元
            self.totalBasePrice = totalBasePrice  # 风电机组基础总价 万元
            self.totalCollectorLinePrice = totalCollectorLinePrice  # 集电线路总价 万元
            # 施工安装平台总价 万元
            self.totalConstructionInstallationPlatformPrice = totalConstructionInstallationPlatformPrice
            self.totalFanPrice = totalFanPrice  # 风机总价 万元
            self.totalInnerRoadPrice = totalInnerRoadPrice  # 场内道路总价 万元
            self.totalPackageSubstationPrice = totalPackageSubstationPrice  # 箱变总价 万元
            self.totalTowerPrice = totalTowerPrice  # 塔筒总价 万元
            # 风机塔筒安装总价 万元
            self.totalTurbineAndTowerInstallationPrice = totalTurbineAndTowerInstallationPrice

    def keys(self):
        if self.enable | self.competition:
            return ('enable', 'turbineNumber', 'baseConcretePrice', 'baseCushionConcretePrice', 'baseSteelBarPrice',
                    'staticTotalInvestment', 'totalBasePrice', 'totalCollectorLinePrice',
                    'totalConstructionInstallationPlatformPrice',
                    'totalFanPrice', 'totalInnerRoadPrice', 'totalPackageSubstationPrice', 'totalTowerPrice')
        else:
            return ('enable', 'turbineNumber')

    def __getitem__(self, item):
        return getattr(self, item)


class PlanParam1(object):
    def __init__(self, competition=False, internetHour=2435, collectorLineLength=None,
                 roadLength=None):
        self.internetHour = internetHour  # 上网小时数
        self.competition = competition
        if competition:  # 竞价添加的参数
            self.collectorLineLength = collectorLineLength  # 集电线路长度 这里传总的 多少个list都传同一个，每个都要写同一个值
            self.roadLength = roadLength  # 场内道路长度 这里传总的 多少个list都传同一个，每个都要写同一个值

    def keys(self):
        if self.competition:
            return ('collectorLineLength', 'roadLength', 'internetHour')
        else:
            return ('internetHour',)

    def __getitem__(self, item):
        return getattr(self, item)


class PlanList(object):

    def __init__(self, competition=False, turbine="GW121/2000", turbinePrice=3500.0, number=22,
                 hubHeight=120, hubType="H", model="GW121/2000-120H", turbineHubHeight=0,
                 concreteAmount=517.0, otherCost=0.0, steelBarAmount=40.0, concreteCushionAmount=30.0,
                 installCost=43.0, mixingTowerCost=79.04, anchorWeight=0.0, steelTowerWeight=192.95834):
        self.competition = competition
        self.turbine = turbine
        self.turbinePrice = turbinePrice  # 风机价格 元/kWh
        self.number = number  # 风机台数
        self.hubHeight = hubHeight
        self.hubType = hubType
        self.model = model  # 机型
        self.turbineHubHeight = turbineHubHeight  # 轮毂高度 m 这个参数应当是由后台查找数据库出来的。
        self.concreteAmount = concreteAmount  # 混凝土用量 立方米/台风机
        self.otherCost = otherCost  # 其他 万元/台
        self.steelBarAmount = steelBarAmount  # 钢筋 t/台
        self.concreteCushionAmount = concreteCushionAmount  # 垫层混凝土用量 立方米／台风机
        self.installCost = installCost  # 安装费 万元／台    # 竞价模式没有此参数
        self.mixingTowerCost = mixingTowerCost  # 混塔段-总价 万元/台
        self.anchorWeight = anchorWeight  # 锚栓重量 t/台
        self.steelTowerWeight = steelTowerWeight  # 钢塔重量 t/台

        self.turbineParams = {'concreteAmount': concreteAmount, "otherCost": otherCost,
                              "steelBarAmount": steelBarAmount,
                              "concreteCushionAmount": concreteCushionAmount}
        if competition:
            self.towerParams = {"mixingTowerCost": mixingTowerCost, "anchorWeight": anchorWeight,
                                "steelTowerWeight": steelTowerWeight, "installCost": installCost}
        else:
            self.towerParams = {"mixingTowerCost": mixingTowerCost, "anchorWeight": anchorWeight,
                                "steelTowerWeight": steelTowerWeight}

    def keys(self):
        if self.competition:
            return ('turbinePrice', 'number', 'turbineHubHeight', 'model', 'turbine', 'hubHeight',
                    'hubType', 'turbineParams', 'towerParams')
        else:
            return ('turbinePrice', 'number', 'turbineHubHeight', 'model', 'turbineParams',
                    'towerParams')

    def __getitem__(self, item):
        return getattr(self, item)


class FinancingExtraInfo(object):
    '''financingExtraInfo {} 其他融资信息'''

    def __init__(self, payInterestTimes=4, debtFinancingRate=0.0435, capitalCurrentInvestmentRatio=0.3,
                 payPrincipalTimes=4, capitalLongInvestmentRatio=0.2,
                 workingCapitalPrice=30.0):
        self.payInterestTimes = payInterestTimes  # 每年付息次数 次
        self.debtFinancingRate = debtFinancingRate  # 短期债务融资利率 % 给数值
        self.capitalCurrentInvestmentRatio = capitalCurrentInvestmentRatio  # 资本金比例-流动资金 % 给数值
        self.payPrincipalTimes = payPrincipalTimes  # 每年还本次数 次
        self.capitalLongInvestmentRatio = capitalLongInvestmentRatio  # 资本金比例-流动资金 % 给数值
        self.workingCapitalPrice = workingCapitalPrice  # 流动资金配置标准 元/kW

    def keys(self):
        return ('payInterestTimes', 'debtFinancingRate', 'capitalCurrentInvestmentRatio',
                'payPrincipalTimes', 'capitalLongInvestmentRatio',
                'workingCapitalPrice')

    def __getitem__(self, item):
        return getattr(self, item)


class ProjectExtraInfo(object):
    ''' projectExtraInfo {} 其他项目信息小类'''

    def __init__(self, competition=False, equityCapitalCost=0.1, approvalDate="2018-01-01",
                 wacc=0.08, operationPeriodMonths=240, startDate="2019-01-01"):
        self.competition = competition
        if competition:
            self.equityCapitalCost = None  # 竞价设置为null
            self.wacc = None
        else:
            self.equityCapitalCost = equityCapitalCost  # 股权资金成本 % 给数值（就是8%==0.08）
            self.wacc = wacc  # 资金成本WACC % 给数值（就是8%==0.08）
        self.approvalDate = approvalDate  # 核准日期
        self.operationPeriodMonths = operationPeriodMonths  # 项目运营期 月
        self.startDate = startDate  # 开工日期

    def keys(self):
        return ('equityCapitalCost', 'approvalDate', 'wacc',
                'operationPeriodMonths', 'startDate')

    def __getitem__(self, item):
        return getattr(self, item)


class OperationExtraInfo(object):
    '''operationExtraInfo {} 其他运维信息'''

    def __init__(self, competition=False, insuranceRate=0.0025, staffNumber=9,
                 wagePlusRate=0.6,
                 inflationMetirail=True, billing="PROPORTION",
                 metierialPrice=[15.0, 15.0, 15.0, 0.0],
                 inflationRepairFee=False, inflationOtherFee=True,
                 repairFeeRate=[0.005, 0.005, 0.005, 0.005, 0.005,
                                0.01, 0.01, 0.01, 0.01, 0.01, 0.015, 0.015,
                                0.015, 0.015, 0.015, 0.02, 0.02, 0.02, 0.02, 0.02],
                 avgWage=8.0, inflationLabor=True, rentSeaUseFee=0.0,
                 inflationBackUpMoney=True, inflationRate=0.03, otherFeePrice=30.0,
                 inflationInsurance=False):
        self.competition = competition
        self.insuranceRate = insuranceRate  # 保险费率 % 给数值
        if competition:
            self.staffNumber = None  # 人员编制 人      #竞价设置为null
        else:
            self.staffNumber = staffNumber
        self.wagePlusRate = wagePlusRate  # 工资附加率 % 传数值
        self.inflationMetirail = inflationMetirail  # 是否考虑通胀-材料费 Y/N
        self.billing = billing  # 计费方式 比例法
        self.metierialPrice = metierialPrice  # 材料费 1-2年、3-5年、6-20年、21-25年四个值
        self.inflationRepairFee = inflationRepairFee  # 是否考虑通胀-修理费 Y/N
        self.inflationOtherFee = inflationOtherFee  # 是否考虑通胀-其他费用 Y/N
        self.repairFeeRate = repairFeeRate  # 修理费 % 给数值 1-25年
        self.avgWage = avgWage  # 平均工资 万元/人
        self.inflationLabor = inflationLabor  # 是否考虑通胀-人工成本 Y/N
        self.rentSeaUseFee = rentSeaUseFee  # 地租/海域使用费 万元／年
        self.inflationBackUpMoney = inflationBackUpMoney  # 是否考虑通胀-备用科目 Y/N
        self.inflationRate = inflationRate  # 是否考虑通胀-通货膨胀率
        self.otherFeePrice = otherFeePrice  # 其他费用 元/kW
        self.inflationInsurance = inflationInsurance  # 是否考虑通胀-保险费率 Y/N

    def keys(self):
        return ('insuranceRate', 'staffNumber', 'wagePlusRate',
                'inflationMetirail', 'billing', 'metierialPrice', 'inflationRepairFee',
                'inflationOtherFee', 'repairFeeRate', 'avgWage', 'inflationLabor',
                'rentSeaUseFee', 'inflationBackUpMoney', 'inflationRate',
                'otherFeePrice', 'inflationInsurance')

    def __getitem__(self, item):
        return getattr(self, item)


class MarketExtraInfo(object):
    ''' marketExtraInfo 其他市场信息'''

    def __init__(self, educationSurtax=0.03, localSubsidyYearNumber=0, constructionTax=0.05,
                 lineLossRatio=0.0,
                 tradingPrice=0.0, incomeSubsidies=[24, 12, 3, 3],
                 incomeBenchmark=1, incomeLineSubsidy=[24, 12, 3, 3],
                 incomeLocalSubsidy=3, tradingRatio=0.0,
                 depreciationAge=20, vatTax=0.16, tradingYearNumber=0,
                 localEducationSurtax=0.02, localSubsidy=0.0, lineSubsidy=0.0,
                 eHour=2490.0, windProductRefundTax=0.5, equipLossRatio=0.0,
                 lineSubsidyYearNumber=20, remainValueRatio=0.05):
        self.educationSurtax = educationSurtax  # 教育费附加 % 给数值
        self.localSubsidyYearNumber = localSubsidyYearNumber  # 地方补贴年限 年
        self.constructionTax = constructionTax  # 城建税 % 给数值
        self.lineLossRatio = lineLossRatio  # 线损 % 给数值
        self.tradingPrice = tradingPrice  # 市场交易电价,单位元/kWh
        self.incomeSubsidies = incomeSubsidies  # 补贴电价部分 第1年 月 第2年 月 第3年 月 第4年及以后 月
        self.incomeBenchmark = incomeBenchmark  # 燃煤标杆部分 月
        self.incomeLineSubsidy = incomeLineSubsidy  # 线路补贴部分 第1年 月 第2年 月 第3年 月 第4年及以后 月
        self.incomeLocalSubsidy = incomeLocalSubsidy  # 地方补贴部分 月
        self.tradingRatio = tradingRatio  # 市场交易电量比例 %，同样，给数值
        self.depreciationAge = depreciationAge  # 折旧年限 年
        self.vatTax = vatTax  # 增值税税率 %给数值
        self.tradingYearNumber = tradingYearNumber  # 市场交易年数 年
        self.localEducationSurtax = localEducationSurtax  # 地方教育费附加 % 给数值
        self.localSubsidy = localSubsidy  # 地方补贴,单位元/kWh
        self.lineSubsidy = lineSubsidy  # 线路补贴,单位元/kWh
        self.eHour = eHour  # 等效小时数 H
        self.windProductRefundTax = windProductRefundTax  # 风电产品即征即退 %给数值
        self.equipLossRatio = equipLossRatio  # 设备损耗 % 给数值
        self.lineSubsidyYearNumber = lineSubsidyYearNumber  # 线路补贴年限 年
        self.remainValueRatio = remainValueRatio  # 残值率 % 给数值

    def keys(self):
        return ('educationSurtax', 'localSubsidyYearNumber', 'constructionTax',
                'lineLossRatio',
                'tradingPrice', 'incomeSubsidies',
                'incomeBenchmark', 'incomeLineSubsidy',
                'incomeLocalSubsidy', 'tradingRatio',
                'depreciationAge', 'vatTax', 'tradingYearNumber',
                'localEducationSurtax', 'localSubsidy', 'lineSubsidy',
                'eHour', 'windProductRefundTax', 'equipLossRatio',
                'lineSubsidyYearNumber', 'remainValueRatio')

    def __getitem__(self, item):
        return getattr(self, item)


class PriceExtraInfo(object):
    '''priceExtraInfo 其他价格信息'''

    def __init__(self, competition=False, equipmentInAndOutFee=50.0,
                 enviromentProtectFee=110.0, equipStorageRate=0.006,
                 innerRoadPrice=35.0,
                 landAcquisitionCompensationFee=2.5, collectorLineUnitPrice=40.0,
                 windTurbineFee=3.0, powerFee=40.0,
                 soilAndWaterConservationFee=220.0, entryRoadPrice=100.0,
                 waterFee=20.0, roadFee=0.0, reconstruRoadPrice=35.0,
                 basicReserveFundsRatio=0.01, surveyAndDesignFee=607.2, landAcquisitionFee=10.0,
                 otherFeeAdjustRatio=1.0):
        self.competition = competition
        if competition:
            self.enviromentProtectFee = None
            self.innerRoadPrice = None
            self.collectorLineUnitPrice = None
            self.surveyAndDesignFee = None
            self.windTurbineFee = None
            self.soilAndWaterConservationFee = None
        else:
            self.enviromentProtectFee = enviromentProtectFee  # 环境保护工程 万元    #竞价设置为null
            self.innerRoadPrice = innerRoadPrice  # 场内道路单价 万元/km         #竞价设置为null
            self.collectorLineUnitPrice = collectorLineUnitPrice  # 集电线路单价 元/km      #竞价设置为null
            self.surveyAndDesignFee = surveyAndDesignFee  # 勘查设计费 万元     #竞价设置为null
            self.windTurbineFee = windTurbineFee  # 风电机组安装平台 基/万元 比如一共5万，这就是5#竞价设置为null
            self.soilAndWaterConservationFee = soilAndWaterConservationFee  # 水土保持工程 万元 #竞价设置为null

        # 大型吊装机械设备进出场费 项/万元 比如一共5万，这就是5
        self.equipmentInAndOutFee = equipmentInAndOutFee
        self.equipStorageRate = equipStorageRate  # 设备采购保管费 % 给数值
        self.landAcquisitionCompensationFee = landAcquisitionCompensationFee  # 征地补偿费 万元/亩
        self.powerFee = powerFee  # 施工电源 项/万元 比如一共5万，这就是5
        self.entryRoadPrice = entryRoadPrice  # 进场道路单价 万元/km
        self.waterFee = waterFee  # 施工水源 项/万元 比如一共5万，这就是5
        self.roadFee = roadFee  # 施工道路 万元 比如一共5万，这就是5
        self.reconstruRoadPrice = reconstruRoadPrice  # 改建道路单价 万元/km
        self.basicReserveFundsRatio = basicReserveFundsRatio  # 基本预备费 % 给数值
        self.landAcquisitionFee = landAcquisitionFee  # 永久征地费用 万元/亩
        self.otherFeeAdjustRatio = otherFeeAdjustRatio  # 其他费用调整 % 给数值

    def keys(self):
        return ('equipmentInAndOutFee', 'enviromentProtectFee', 'equipStorageRate',
                'innerRoadPrice', 'landAcquisitionCompensationFee', 'collectorLineUnitPrice',
                'windTurbineFee', 'powerFee', 'soilAndWaterConservationFee', 'entryRoadPrice',
                'waterFee', 'roadFee', 'reconstruRoadPrice',
                'basicReserveFundsRatio', 'surveyAndDesignFee', 'landAcquisitionFee',
                'otherFeeAdjustRatio')

    def __getitem__(self, item):
        return getattr(self, item)


class EngineeringExtraInfo(object):
    '''engineeringExtraInfo 其他工程信息'''

    def __init__(self, competition=False, innerRoadWidth=6.0,
                 combinedRunningCostRate=0.004, boosterArea=10000.0,
                 projectInsuranceRate=0.004, sparePartsFeeRate=0.003,
                 specialReportPreparationRate=0.0,
                 projectPreConstructionRate=0.01, entryRoadLength=0.5):
        self.competition = competition
        if competition:
            self.boosterArea = None  # 竞价设置为null
        else:
            self.boosterArea = boosterArea  # 升压站占地
        self.innerRoadWidth = innerRoadWidth  # 场内道路宽度 m
        self.combinedRunningCostRate = combinedRunningCostRate  # 联合试运转费 % 给数值
        self.projectInsuranceRate = projectInsuranceRate  # 工程保险费 % 给数值
        self.sparePartsFeeRate = sparePartsFeeRate  # 备品备件购置费 % 给数值
        self.specialReportPreparationRate = specialReportPreparationRate  # 专题报告编制费 % 给数值
        self.projectPreConstructionRate = projectPreConstructionRate  # 工程前期费 % 给数值
        self.entryRoadLength = entryRoadLength  # 进场道路(长度) km

    def keys(self):
        return ('innerRoadWidth', 'combinedRunningCostRate', 'boosterArea',
                'projectInsuranceRate', 'sparePartsFeeRate',
                'specialReportPreparationRate',
                'projectPreConstructionRate', 'entryRoadLength')

    def __getitem__(self, item):
        return getattr(self, item)


class EconomicComp(ExplicitComponent):

    def __init__(self):
        super(EconomicComp, self).__init__()

        self.add_param('turbinePrice', val=0.0)
        self.add_param('internetHour', val=0.0)
        self.add_param('concreteAmount', val=0.0)
        self.add_param('steelTowerWeight', val=0.0)

        self.add_output('lcoe', val=0.0)
        self.add_output('eirr', val=0.0)

        
turbines = {"GW140/2500":{'turbinePrice':3500.0,"Towerweight":174,
                            "steelBarAmount":38.00,"concreteAmount":525,
                            "lowerhour":3197,"upperhour":3365},
                "GW140/3400":{'turbinePrice':3600.0,"Towerweight":202,
                            "steelBarAmount":48.00,"concreteAmount":579,
                            "lowerhour":2874,"upperhour":1933}}


def optimize_economic(model, internetHour, numberTurbine = [5,10]):
    
    if len(model) != len(numberTurbine):
        return {'error': "lengthes are not equal!"}
    
    searchable = False  # 可研
    competition = False  # 竞价
    distribut_en = False  # 分散式
    operation_en = False  # 代运维
    
    projectParam = {}
    projectExtra = {}
    planParam = dict(PlanParam1(competition=True, internetHour=internetHour))
    planList1 = {}
    for i,turbine in enumerate(model):
        if "140/2500" in turbine:
            turbineChoose = turbines["GW140/2500"]
            planList1.update(dict(PlanList(competition=True, turbinePrice=turbineChoose['turbinePrice'],
                                      concreteAmount=turbineChoose['concreteAmount'],
                              steelTowerWeight=turbineChoose['Towerweight'],
                                      steelBarAmount=turbineChoose['steelBarAmount'],
                                          number=numberTurbine[i])))
        if "140/3400" in turbine:
            turbineChoose = turbines["GW140/3400"]
            planList1.update(dict(PlanList(competition=True, 
                                           turbinePrice=turbineChoose['turbinePrice'],
                                      concreteAmount=turbineChoose['concreteAmount'],
                              steelTowerWeight=turbineChoose['Towerweight'],
                                      steelBarAmount=turbineChoose['steelBarAmount'],
                                           number=numberTurbine[i])))
    
    planParam.update({"planList":[planList1]})
            

    

    distributInfo1 = dict(DistributInfo(competition=True))
    priceInfo1 = dict(PriceInfo())
    financingInfo1 = dict(FinancingInfo())
    projectInfo1 = dict(ProjectInfo())
    operationInfo1 = dict(OperationInfo(competition=True))
    engineeringInfo1 = dict(EngineeringInfo())
    marketInfo1 = dict(MarketInfo())
    feasibleInfo1 = dict(FeasibleInfo(competition=True))
    financingExtraInfo1 = dict(FinancingExtraInfo())
    projectExtraInfo1 = dict(ProjectExtraInfo(competition=True))
    operationExtraInfo1 = dict(OperationExtraInfo(competition=True))
    marketExtraInfo1 = dict(MarketExtraInfo())
    priceExtraInfo1 = dict(PriceExtraInfo(competition=True))
    engineeringExtraInfo1 = dict(EngineeringExtraInfo(competition=True))

    projectParam.update({'distributInfo': distributInfo1})
    projectParam.update({'priceInfo': priceInfo1})
    projectParam.update({'financingInfo': financingInfo1})
    projectParam.update({'projectInfo': projectInfo1})
    projectParam.update({'operationInfo': operationInfo1})
    projectParam.update({'engineeringInfo': engineeringInfo1})
    projectParam.update({'marketInfo': marketInfo1})
    projectParam.update({'feasibleInfo': feasibleInfo1})
    
    projectExtra.update({'financingExtraInfo': financingExtraInfo1})
    projectExtra.update({'projectExtraInfo': projectExtraInfo1})
    projectExtra.update({'operationExtraInfo': operationExtraInfo1})
    projectExtra.update({'marketExtraInfo': marketExtraInfo1})
    projectExtra.update({'priceExtraInfo': priceExtraInfo1})
    projectExtra.update({'engineeringExtraInfo': engineeringExtraInfo1})

    data = {'projectParam': projectParam, 'planParam': planParam, 'projectExtra': projectExtra}
    headers = {'Content-Type': 'application/json'}
    url = "http://wind-evaluation-2-1-dev.api-general.wrinternal.goldwind.com.cn/windmodel"

    dataopen = json.dumps(data)  # , ensure_ascii=False
    data_encode = dataopen.encode("utf-8")
    r = requests.post(url, data=data_encode, headers=headers)

    data_r = json.loads(r.text)
#     print(r.text)
    lcoe = data_r['dataRows']['bidScheme']['lcoe']
    eirr = data_r['dataRows']['bidScheme']['eirr']

    return lcoe, eirr
optimize_economic(model = ["GW140/3400-120H","GW140/3400-120H"],internetHour = 2300,numberTurbine = [5,10])

(0.4758323943926956, 0.05645828247070314)

In [21]:
from sympy import *
x=Symbol('x')
y=Symbol('y')
print (solve(1.5*y+2.2*x-50,x,y))

[{x: -0.681818181818182*y + 22.7272727272727}]


In [48]:
import numpy as np
xList = []
yList = []
for a in np.arange(0,100):
    for b in np.arange(0,100) :
        l = np.arange(48.0,52.0,0.1)
#         print(l)
        if float(2.2*a+1.5*b) in l:
            xList.append(a)
            yList.append(b)
print(xList)
print(yList)

import itertools
print(list(itertools.combinations([1,2],2)))

[0, 7, 11, 13, 15, 22]
[32, 22, 16, 13, 10, 0]
[(1, 2)]
